In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from astropy.visualization import quantity_support
quantity_support()

In [ ]:
import matplotlib.pyplot as plt

import numpy as np

from astropy.time import Time
from astropy.coordinates import get_body_barycentric, SkyCoord
import astropy.units as u

from sunpy.coordinates import get_body_heliographic_stonyhurst
import sunpy.map

import astropy.constants as const
import scipy
from sunpy.coordinates import frames

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,8)

In [ ]:
#!rm ~sunpy/data/secchi_l0_a_img_euvi_20100819_20100819_000715_n4eua.fts
#!rm ~/sunpy/data/aia_lev1_171a_2010_08_19t00_00_00_34z_image_lev1.fits

# SunPy & Coordinates

SunPy coordinates allows us to deal with points in physical space, and the SunPy coordinates subpackage provides definitions of and transformations between several reference frames commonly used in solar physics. This allows us to manipulate Maps and take advantage of WCS Axes for plotting

These reference frames and their associated transformations are implemented using the `astropy.coordinates` subpackage and extend Astropy’s coordinate frame transformation graph to include solar coordinates

# STEREO & SDO loop 
In this example we will see how powerful the coordinates functionality in SunPy is for performing SDO-STEREO data analysis 

Lets first download some AIA and STEREO data

In [ ]:
from sunpy.net import Fido, attrs as a


stereo = (a.vso.Source('STEREO_A') &
          a.Instrument('EUVI') &
          a.Time('2010-08-19', '2010-08-19T00:10:00'))

aia = (a.Instrument('AIA') &
       a.vso.Sample(24 * u.hour) &
       a.Time('2010-08-19', '2010-08-19T00:10:00'))

wave = a.Wavelength(17 * u.nm, 18 * u.nm)


res = Fido.search(wave, aia | stereo)

In [ ]:
files = Fido.fetch(res)

### Now lets make maps for both AIA and EUVI 

In [ ]:
files

In [ ]:
map_aia, map_stereo = sunpy.map.Map(sorted(files))

### Compare the Locations of STEREO-A and AIA

In [ ]:
sun_pos = get_body_heliographic_stonyhurst('sun', time = map_aia.date)

In [ ]:
ax = plt.subplot(projection='polar')

plt.polar(map_stereo.observer_coordinate.lon.to(u.rad), map_stereo.observer_coordinate.radius.to(u.AU), 
          marker='o',  ms=10, label='STEREO_A')
plt.polar(map_aia.observer_coordinate.lon.to(u.rad), map_aia.observer_coordinate.radius.to(u.AU), 
          marker='o', ms=10, label='AIA')

plt.polar(sun_pos.lon.to(u.rad), sun_pos.radius.to(u.AU), 'o', ms=20, label='Sun', color='yellow')
ax.set_theta_zero_location("S")

plt.title('Position of the Sun, AIA and Stereo')
plt.legend()

And plot the two images together

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(1,2,1, projection = map_aia)
map_aia.plot(axes = ax1)
ax2 = fig.add_subplot(1,2,2, projection = map_stereo)
map_stereo.plot(axes = ax2)

### Lets now focus on the active region that is at ~ (500'', 480'') in AIA that is also seen in EUVI
To do this we can define a box in AIA and then transform the corners of the box to the coordinate system of EUVI and overplot a rectange on both. 

In [ ]:
bl_aia = SkyCoord(350*u.arcsec, 300*u.arcsec, frame=map_aia.coordinate_frame)
width = 300*u.arcsec
height = 280*u.arcsec
corners_aia = ([bl_aia.Tx, bl_aia.Ty],
               [bl_aia.Tx + width, bl_aia.Ty],
               [bl_aia.Ty, bl_aia.Ty + height],
               [bl_aia.Tx + width, bl_aia.Ty + height])

corners_aia = SkyCoord(corners_aia, frame=map_aia.coordinate_frame)

#the coordinates of the corners now in STEREO/EUVI coordinate system
corners_stereo = corners_aia.transform_to(map_stereo.coordinate_frame)

In [ ]:
fig = plt.figure()
ax = plt.subplot(1,2,1, projection=map_aia)
map_aia.plot(axes=ax)
map_aia.draw_rectangle(corners_aia[0], 
                       corners_aia[1].Tx - corners_aia[0].Tx, 
                       corners_aia[2].Ty - corners_aia[0].Ty)

ax2 = plt.subplot(1,2,2, projection=map_stereo)
map_stereo.plot(axes=ax2)
map_stereo.draw_rectangle(corners_stereo[0], 
                          corners_stereo[1].Tx - corners_stereo[0].Tx, 
                          corners_stereo[2].Ty - corners_stereo[0].Ty)



### Lets define some coordinates of a semicirclar loop of the active region and then plot this in both fields of view

In [ ]:
@u.quantity_input
def semi_circular_loop(length: u.m, lon: u.deg=0*u.deg, lat: u.deg=0*u.deg):
    """
    Return a Heliographic Stonyhurst coordinate object with points of a semi circular loop in it.
    
    Parameters
    ----------
    length : `~astropy.Quantity`
        Distance between footpoints of loops
    lon : ~astropy.Quantity`
        Longitude value of center of the loop
    lat : `~astropy.Quantity`
        Latitude value of center of loop
    
    Returns
    -------
    
    `astopy.coordinate.SkyCoord`
        A SkyCoord of the coordinates of a semi-circular loop
    
    """
    r_sun = const.R_sun

    def r_2_func(x):
        return np.arccos(0.5 * x / r_sun.to(u.cm).value) - np.pi + length.to(u.cm).value / 2. / x

    r_2 = scipy.optimize.bisect(r_2_func,
                                length.to(u.cm).value / (2 * np.pi),
                                length.to(u.cm).value / np.pi) * u.cm

    alpha = np.arccos(0.5 * (r_2 / r_sun).decompose())
    phi = np.linspace(-np.pi * u.rad + alpha, np.pi * u.rad - alpha, 2000)

    # Quadratic formula to find r
    a = 1.
    b = -2 * (r_sun.to(u.cm) * np.cos(phi.to(u.radian)))
    c = r_sun.to(u.cm)**2 - r_2.to(u.cm)**2
    r = (-b + np.sqrt(b**2 - 4 * a * c)) / 2 / a
    # Choose only points above the surface
    i_r = np.where(r > r_sun)
    r = r[i_r]
    phi = phi[i_r]
    hcc_frame = frames.Heliocentric(
        observer=SkyCoord(lon=lon, lat=lat, radius=r_sun, frame='heliographic_stonyhurst'))

    return SkyCoord(
        x=r.to(u.cm) * np.sin(phi.to(u.radian)),
        y=u.Quantity(r.shape[0] * [0 * u.cm]),
        z=r.to(u.cm) * np.cos(phi.to(u.radian)),
        frame=hcc_frame).transform_to('heliographic_stonyhurst')


We can create a loop structure that fits over the active region of choice and plot it on AIA

In [ ]:
# here we are defining the loop coordinates
loop_coords = semi_circular_loop(200*u.Mm, lon=38*u.deg, lat=31*u.deg)

# Make a map the size of our rectangle
sub_aia = map_aia.submap(corners_aia[0], corners_aia[3])

In [ ]:
fig = plt.figure()

ax = plt.subplot(1,2,1, projection=map_aia)
map_aia.plot(axes=ax)
map_aia.draw_rectangle(corners_aia[0], 
                       corners_aia[1].Tx - corners_aia[0].Tx, 
                       corners_aia[2].Ty - corners_aia[0].Ty)

ax.plot_coord(loop_coords.transform_to(map_aia.coordinate_frame), color='r', lw=2)

ax1 = plt.subplot(1,2,2, projection=sub_aia)

sub_aia.plot(axes=ax1)
ax1.plot_coord(loop_coords.transform_to(sub_aia.coordinate_frame), color='r', lw=2)

### Plot the Loop on Both submaps

In [ ]:
sub_stereo = map_stereo.submap(corners_stereo[0], corners_stereo[3])

In [ ]:
ax = plt.subplot(1,2,1, projection = sub_aia)
sub_aia.plot(axes = ax)
ax.plot_coord(loop_coords.transform_to(sub_aia.coordinate_frame), color = 'red', lw = 2)


ax2 = plt.subplot(1,2,2, projection = sub_stereo)
sub_stereo.plot(axes = ax2)
ax2.plot_coord(loop_coords.transform_to(sub_stereo.coordinate_frame), color = 'red', lw = 2)

### Extract the Intensity Values along the Loop 

In [ ]:
x, y = np.int64(u.Quantity(map_aia.world_to_pixel(loop_coords)))

aia_loop_int = map_aia.data[y, x]

In [ ]:
x, y = np.int64(u.Quantity(map_stereo.world_to_pixel(loop_coords)))

stereo_loop_int = map_stereo.data[y, x]

In [ ]:
distance = np.cumsum(loop_coords[:-1].separation_3d(loop_coords[1:])).insert(0, 0*u.km).to(u.Mm)

In [ ]:
l_stereo, = plt.plot(distance, stereo_loop_int)
plt.ylabel("Intensity (STEREO)")
plt.xlabel("Distance Along Loop [Mm]")
plt.twinx()

l_aia, = plt.plot(distance, aia_loop_int, color="C1")
plt.ylabel("Intensity (AIA)")

plt.legend((l_stereo, l_aia), ("STEREO", "AIA"))